# <span style="color:#72A98F">Location from Timestamp Notebook</span>

Hello Person! In this notebook I will be using the *[LLAT](../../Data/Location/LIGHT-1_LLA_April.csv)* (Latitude Longitude Altitude Time) file to find the location at a given **Event Timestamp**.

## <span style="color:#72A98F">Importing libraries and giving constant values</span>

In [550]:
import numpy as np
import pandas as pd
from astropy.time import Time
import raadpy as rp
import os
from datetime import datetime
import calendar

#Random Values from March to April
TSA = np.random.uniform(low=59639.000000000, high=59700.00000000, size=100)

#Random Values in March alone
# TSA = np.random.uniform(low=59639.000000000, high=59670.000000000, size=10)


print(TSA)

[59678.48507599 59660.22503572 59680.7529531  59640.46854309
 59699.05686317 59663.64185502 59698.63756515 59655.30442825
 59651.70049262 59678.38364635 59644.12207649 59686.61490206
 59639.65815073 59696.31337055 59679.13745842 59698.52329381
 59664.07165138 59682.28423664 59642.11299305 59657.6979114
 59675.57190001 59685.54646937 59661.60020785 59666.11568572
 59685.16154879 59675.00730588 59685.81349888 59687.77986764
 59665.04174617 59647.75696875 59664.69960179 59654.42235472
 59658.31059699 59686.99028133 59643.93364677 59685.94241153
 59649.21596758 59688.73677833 59686.32844729 59642.21398576
 59664.6360992  59659.28033657 59685.93947833 59666.43314172
 59685.74991019 59652.84380055 59653.28844601 59662.36843412
 59674.88477252 59643.0732479  59646.01863218 59688.85920744
 59651.81781238 59688.4358495  59671.87762289 59639.18239765
 59657.31357518 59676.18058897 59678.86595932 59673.54658931
 59681.16889873 59684.24190701 59691.74496711 59678.77080081
 59694.94675921 59686.661

## <span style="color:#72A98F">Function For Finding Timestamp Locations for an Array of Timestamps

In [565]:
def Location_from_Timestamp(TimestampArray, LocationFolder):

    # type(TimestampArray)
    arr = os.listdir(LocationFolder)

    months = [calendar.month_name[i] for i in range(1, 13)]
    FilesFound = []

    Columns = ['Time','Lon','Lat']
    ColumnTitles  = {}
    ColumnNumbers = {}

    MinTSMonth,MaxTSMonth = int(Time(np.amin(TSA), format='mjd').iso[6]),int(Time(np.amax(TSA), format='mjd').iso[6])
    FilesRequired = months[MinTSMonth-1:MaxTSMonth]
    print('Files Required are:', FilesRequired)

    for m in FilesRequired:
        k = [(i) for i in arr if m in i ]
        FilesFound.append(k)

    
    FileData={}

    for i in range(len(FilesRequired)):
        if len(FilesFound[i]) != 0:
            print(FilesRequired[i] + " file is found and opened!", flush=True)
            FileData[FilesRequired[i]] = pd.read_csv('../../Data/Location/'+FilesFound[i][0])
            
        else:
            print(FilesRequired[i] + " file is NOT found, this time range will not be searched! ＼(º □ º l|l)/", flush=True)
    
    print('')

    for c in range(len(Columns)):

        Ckey = [[b,c] for b in FileData[[[FilesRequired[u]] for u in range(len(FilesFound)) if len(FilesFound[u]) != 0][0][0]].keys() if Columns[c] in b ]
        ColumnNumbers[Columns[c]] = Ckey[0][1]
        ColumnTitles[Columns[c]]  = Ckey[0][0]
            
    # print('Column Dictionary Numbers:', ColumnNumbers)
    # print('Column Dictionary Names:', ColumnTitles, '\n')
    

    #Part 2
    Values = {}

    for i in range(len(FilesRequired)):

        if len(FilesFound[i]) != 0:
            MonthStart = datetime.strptime(FilesRequired[i], '%B').month
            MonthEnd   = datetime.strptime(FilesRequired[i], '%B').month + 1

            Values[FilesRequired[i]] = np.where(np.logical_and(TimestampArray <= Time('2022-'+str(MonthEnd)+'-01').mjd, TimestampArray >= Time('2022-'+str(MonthStart)+'-01').mjd))[0]

        else:
            continue
    

    #Part 3
    TSL=[]

    print('Finding',len(TimestampArray) ,'Values!\n', flush=True)

    for i in range(len(FilesFound)):

        if len(FilesFound[i]) != 0:

            for j in Values[FilesRequired[i]]:

                ts = np.float64(TimestampArray[j])

                ResultMin = FileData[FilesRequired[i]][FileData[FilesRequired[i]][ColumnTitles[Columns[0]]] <= ts].nlargest(1, ColumnTitles[Columns[0]])
                ResultMax = FileData[FilesRequired[i]][FileData[FilesRequired[i]][ColumnTitles[Columns[0]]] >= ts].nsmallest(1, ColumnTitles[Columns[0]])
                
                TimeRange = [np.float64(ResultMin.iloc[:,(ColumnNumbers['Time'])]), np.float64(ResultMax.iloc[:,(ColumnNumbers['Time'])])]
                LongRange = [np.float64(ResultMin.iloc[:,(ColumnNumbers['Lon'])]), np.float64(ResultMax.iloc[:,(ColumnNumbers['Lon'])])]
                LatRange  = [np.float64(ResultMin.iloc[:,(ColumnNumbers['Lat'])]), np.float64(ResultMax.iloc[:,(ColumnNumbers['Lat'])])]
                longitude = np.interp(ts, TimeRange, LongRange)
                latitude = np.interp(ts, TimeRange, LatRange)
                TSL.append([ts,longitude,latitude])
        
        else:
            continue

    print(TSL)

    print("\nDone! (￣^￣ )ゞ")

In [566]:
Location_from_Timestamp(TimestampArray = TSA, LocationFolder = '../../Data/Location/')

Files Required are: ['March', 'April']
March file is found and opened!
April file is found and opened!

Finding 100 Values!

[[59660.22503572161, 51.558394537250955, -116.57343702683495], [59640.46854308693, 23.53412165122072, -24.122611513618992], [59663.641855016984, 39.19133399034773, 115.59395622047283], [59655.304428245945, -45.48398977089214, 93.4932262547102], [59651.700492617536, -42.06476931568924, -19.703786485190786], [59644.12207649006, 38.22435282790697, -42.800061937441875], [59639.65815072632, -42.97842914962444, 65.33058704489085], [59664.07165138344, 8.469661657345855, -175.05085330741574], [59642.11299304821, -14.595042248308717, -77.6976329536808], [59657.697911400326, 0.46379160537918235, -8.88438239620293], [59661.60020784564, -34.77419986944776, -144.20630249256683], [59666.11568571878, -49.307829345811676, 83.9479137248934], [59665.04174617022, 28.631962905180696, -151.35229957720276], [59647.75696875046, -25.240810184833208, -129.51549045171888], [59664.69960179

## <span style="color:#72A98F">*Ignore what's below this point*</span>

### Notes

In [3]:
#NOTES

# time = E.get_mjd()

# def f(EV:rp.event):
#     time = EV.get_mjd()

# k = [(i) for i in arr if "June" in i ]
# k

# def g(a):
#     return a+

# import numpy as np
# x = np.random.rand()
# y = np.random.rand(10)
# np.where(x < y)[0]

### Old Code

In [2]:
# # GivenTimestamp = E.get_mjd()
# # A remake of this line above will be used to actually get the timestamp ^
# # But for now, we will name a value that exists in the range of april.
# GivenTimestamp = 59639

# #If the timestamp lies somewhere in March this will use the March file!
# if Time(59639.00000000, format='mjd') <= Time(GivenTimestamp, format='mjd') <= Time(59670.00000000, format='mjd'):
#     print("This time lies in March!")
#     LocationFile = '../../Data/Location/LIGHT-1_LLA_March.csv'
#     LLATData = pd.read_csv(LocationFile)
    
# #If the timestamp lies somewhere in April this will use the April file!
# elif Time(59670.00000000, format='mjd') <= Time(GivenTimestamp, format='mjd') <= Time(59700.00000000, format='mjd'):
#     print("This time lies in April!")
#     LocationFile = '../../Data/Location/LIGHT-1_LLA_April.csv'
#     LLATData = pd.read_csv(LocationFile)
    
# #If the timestamp reached does not exist in the range useable, this will skip that specific value only.
# else:
#     print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.0")

# #Read the time column only
# TData = LLATData["Time (ModJDate)"]

# #Count the rows and look through them.
# for row in range(len(TData)):

#     #
#     if GivenTimestamp == TData[row]:
#         MaximumData = LLATData.T[row]
#         MinimumData = LLATData.T[row]
#         break
    
#     if TData[row] >= GivenTimestamp:
#         MaximumData = LLATData.T[row]
#         MinimumData = LLATData.T[row-1]
#         break

# TimeRange = [MinimumData[0], MaximumData[0]]
# LongRange = [MinimumData[2], MaximumData[2]]
# LatRange  = [MinimumData[1], MaximumData[1]]
# longitude2 = np.interp(GivenTimestamp, TimeRange, LongRange)
# latitude2 = np.interp(GivenTimestamp, TimeRange, LatRange)

# TS = [] #For the values to be added together.
# TS.append([GivenTimestamp, longitude2, latitude2])
# TS

This time lies in March!


[[59639, -157.984, -31.383]]

In [24]:
# # TimestampArray  = [59621,59671]

# TSL = []
# np.where(np.logical_and(TimestampArray <= 59700.00000000, TimestampArray >= 59639.000000000))[0]

# for ts in TimestampArray:

#     if Time('2022-03-01') <= Time(ts, format='mjd') <= Time('2022-03-01'):
#         LLATData = FileData['March']
        
    
#     elif Time(59670.00000000, format='mjd') <= Time(ts, format='mjd') <= Time(59700.00000000, format='mjd'):
#         LLATData = FileData['April']
    
#     else:
#         print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.0")
#         continue

#     TData = LLATData["Time (ModJDate)"]

#     for row in range(len(TData)):

#         if ts == TData[row]:
#             MaximumData = LLATData.T[row]
#             MinimumData = LLATData.T[row]
#             break
        
#         if TData[row] >= ts:
#             MaximumData = LLATData.T[row]
#             MinimumData = LLATData.T[row-1]
#             break
    
#     TimeRange = [MinimumData[0], MaximumData[0]]
#     LongRange = [MinimumData[2], MaximumData[2]]
#     LatRange  = [MinimumData[1], MaximumData[1]]
#     longitude = np.interp(ts, TimeRange, LongRange)
#     latitude = np.interp(ts, TimeRange, LatRange)
#     TSL.append([ts,longitude,latitude])

# print(TSL)

NameError: name 'TSA' is not defined

In [3]:
# TimestampArray  = [59621,59671]
# TSL = []

# for ts in TimestampArray:

#     #If the timestamp lies somewhere in March this will use the March file!
#     if Time(59639.000000000, format='mjd') <= Time(ts, format='mjd') <= Time(59670.00000000, format='mjd'):
#         LocationFile = '../../Data/Location/LIGHT-1_LLA_March.csv'
#         LLATData = pd.read_csv(LocationFile)
        
    
#     #If the timestamp lies somewhere in April this will use the April file!
#     elif Time(59670.00000000, format='mjd') <= Time(ts, format='mjd') <= Time(59700.00000000, format='mjd'):
#         LocationFile = '../../Data/Location/LIGHT-1_LLA_April.csv'
#         LLATData = pd.read_csv(LocationFile)
    
#     #If the timestamp reached does not exist in the range useable, this will skip that specific value only.
#     else:
#         print("The time value given does not exist in the range from March to April!\nFor reference, the time in mjd should be between 59639.0 and 59700.0")
#         continue #Skips the values not in the range!

#     #Telling it to take the time column
#     TData = LLATData["Time (ModJDate)"]

#     #Start the loop that looks through the file.
#     for row in range(len(TData)):

#         #If the timestamp exists in the data file, use the location already given, and leave the loop. (Condition 1)
#         if ts == TData[row]:
#             MaximumData = LLATData.T[row]
#             MinimumData = LLATData.T[row]
#             break
        
#         #If the timestamp does not exist in the file, then take the values before and after it, then leave the loop. (Condition 2)
#         if TData[row] >= ts:
#             MaximumData = LLATData.T[row]
#             MinimumData = LLATData.T[row-1]
#             break
    
#     #If Condition 1 is met, this part will not affect any values.
#     #Otherwise, if Condition 2 is met, this part will interpolate the values linearly. 
#     #(Hopefully using different interpolation methods later!)
#     TimeRange = [MinimumData[0], MaximumData[0]]
#     LongRange = [MinimumData[2], MaximumData[2]]
#     LatRange  = [MinimumData[1], MaximumData[1]]
#     longitude = np.interp(ts, TimeRange, LongRange)
#     latitude = np.interp(ts, TimeRange, LatRange)

#     #Append the values into an array.
#     TSL.append([ts,longitude,latitude])

# print(TSL)

The time value given does not exist in the range from March to April!
For reference, the time in mjd should be between 59639.0 and 59700.0


KeyError: 'Time (ModJDate)'